# Monkeypox Image Classification Using Transfer Learning

This notebook demonstrates how to classify images as "Monkeypox" or "Non Monkeypox" using transfer learning with popular deep learning models (ResNet50, VGG16, and MobileNetV2). The workflow includes data preparation, model training, evaluation, and saving the trained models for future use.

## Data Preparation and Splitting

Setting up the dataset structure and splitting data into train, validation, and test sets.

In [ ]:
import tensorflow as tf
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Paths to the source folders
src_folders = ['dataset/Augmented Images', 'dataset/Original Images']

# Paths to the destination folders
train_folder = 'dataset/train_data'
validation_folder = 'dataset/validation_data'
test_folder = 'dataset/test_data'

# Create necessary directories
for folder in [train_folder, validation_folder, test_folder]:
    for label in ['Monkeypox', 'Non Monkeypox']:
        os.makedirs(os.path.join(folder, label), exist_ok=True)

# Combine data and split into train, validation, and test sets
def split_data(src_folders, train_folder, validation_folder, test_folder):
    for label in ['Monkeypox', 'Non Monkeypox']:
        all_files = []
        for folder in src_folders:
            src_dir = os.path.join(folder, label)
            files = [os.path.join(src_dir, file_name) for file_name in os.listdir(src_dir)]
            all_files.extend(files)
        
        train_files, test_files = train_test_split(all_files, test_size=0.2, random_state=42)
        train_files, val_files = train_test_split(train_files, test_size=0.25, random_state=42)  # 20% of the original dataset for validation
        
        for file_path in train_files:
            shutil.copy(file_path, os.path.join(train_folder, label))
        
        for file_path in val_files:
            shutil.copy(file_path, os.path.join(validation_folder, label))
        
        for file_path in test_files:
            shutil.copy(file_path, os.path.join(test_folder, label))

# Split the data
split_data(src_folders, train_folder, validation_folder, test_folder)

# Load data for training, validation, and testing
train_datagen = ImageDataGenerator(rescale=1./255)
test_val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

validation_generator = test_val_datagen.flow_from_directory(
    validation_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

test_generator = test_val_datagen.flow_from_directory(
    test_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')


Found 2052 images belonging to 2 classes.
Found 684 images belonging to 2 classes.
Found 684 images belonging to 2 classes.


## Model Architecture Setup

Creating transfer learning models using pre-trained architectures.

### ResNet50 Model

In [5]:
# ResNet50

def create_resnet_model():
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

resnet_model = create_resnet_model()




94765736/94765736 [==============================] - 20s 0us/step



### VGG16 Model

In [6]:
# VGG16

def create_vgg16_model():
    base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

vgg16_model = create_vgg16_model()


58889256/58889256 [==============================] - 7s 0us/step


### MobileNetV2 Model

In [7]:
# MobileNetV2

def create_mobilenetv2_model():
    base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

mobilenetv2_model = create_mobilenetv2_model()


## Model Training

Training each model for 10 epochs using the prepared data generators.

In [9]:
#Train ResNet50

resnet_history = resnet_model.fit(
    train_generator, epochs=10, validation_data=validation_generator
)


Epoch 1/10


65/65 [==============================] - 137s 2s/step - loss: 0.7195 - accuracy: 0.5439 - val_loss: 0.6825 - val_accuracy: 0.5570
Epoch 2/10
65/65 [==============================] - 127s 2s/step - loss: 0.6792 - accuracy: 0.5741 - val_loss: 0.6716 - val_accuracy: 0.6140
Epoch 3/10
65/65 [==============================] - 128s 2s/step - loss: 0.6704 - accuracy: 0.6058 - val_loss: 0.6620 - val_accuracy: 0.6140
Epoch 4/10
65/65 [==============================] - 124s 2s/step - loss: 0.6591 - accuracy: 0.6316 - val_loss: 0.6531 - val_accuracy: 0.6623
Epoch 5/10
65/65 [==============================] - 123s 2s/step - loss: 0.6574 - accuracy: 0.6189 - val_loss: 0.6496 - val_accuracy: 0.6754
Epoch 6/10
65/65 [==============================] - 124s 2s/step - loss: 0.6498 - accuracy: 0.6194 - val_loss: 0.6481 - val_accuracy: 0.6769
Epoch 7/10
65/65 [==============================] - 124s 2s/step - loss: 0.6443 - accuracy: 0.6506 - val_loss: 0.6431 - val_accuracy: 0.6272
Epoch 8/10


In [10]:
#Train VGG16

vgg16_history = vgg16_model.fit(
    train_generator, epochs=10, validation_data=validation_generator
)


Epoch 1/10
65/65 [==============================] - 330s 5s/step - loss: 0.6737 - accuracy: 0.6092 - val_loss: 0.6509 - val_accuracy: 0.6681
Epoch 2/10
65/65 [==============================] - 333s 5s/step - loss: 0.6366 - accuracy: 0.6735 - val_loss: 0.6271 - val_accuracy: 0.6798
Epoch 3/10
65/65 [==============================] - 331s 5s/step - loss: 0.6172 - accuracy: 0.6798 - val_loss: 0.6139 - val_accuracy: 0.6871
Epoch 4/10
65/65 [==============================] - 330s 5s/step - loss: 0.6058 - accuracy: 0.6813 - val_loss: 0.6048 - val_accuracy: 0.6813
Epoch 5/10
65/65 [==============================] - 341s 5s/step - loss: 0.5980 - accuracy: 0.6949 - val_loss: 0.5958 - val_accuracy: 0.6944
Epoch 6/10
65/65 [==============================] - 357s 5s/step - loss: 0.5906 - accuracy: 0.6964 - val_loss: 0.5884 - val_accuracy: 0.7003
Epoch 7/10
65/65 [==============================] - 333s 5s/step - loss: 0.5839 - accuracy: 0.7003 - val_loss: 0.5845 - val_accuracy: 0.6974
Epoch 8/10
65

In [11]:
#Train MobileNetV2

mobilenetv2_history = mobilenetv2_model.fit(
    train_generator, epochs=10, validation_data=validation_generator
)


Epoch 1/10
65/65 [==============================] - 31s 438ms/step - loss: 0.5704 - accuracy: 0.6920 - val_loss: 0.4401 - val_accuracy: 0.8202
Epoch 2/10
65/65 [==============================] - 28s 427ms/step - loss: 0.4151 - accuracy: 0.8265 - val_loss: 0.3817 - val_accuracy: 0.8523
Epoch 3/10
65/65 [==============================] - 28s 431ms/step - loss: 0.3605 - accuracy: 0.8548 - val_loss: 0.3601 - val_accuracy: 0.8421
Epoch 4/10
65/65 [==============================] - 28s 425ms/step - loss: 0.3272 - accuracy: 0.8748 - val_loss: 0.3251 - val_accuracy: 0.8874
Epoch 5/10
65/65 [==============================] - 28s 431ms/step - loss: 0.2971 - accuracy: 0.8933 - val_loss: 0.3134 - val_accuracy: 0.8889
Epoch 6/10
65/65 [==============================] - 28s 428ms/step - loss: 0.2770 - accuracy: 0.9030 - val_loss: 0.3032 - val_accuracy: 0.8830
Epoch 7/10
65/65 [==============================] - 28s 429ms/step - loss: 0.2631 - accuracy: 0.9055 - val_loss: 0.2895 - val_accuracy: 0.8977

## Model Saving

Saving the trained models for future use.

In [12]:
resnet_model.save('resnet50_monkeypox_model.h5')


c:\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
vgg16_model.save('vgg16_monkeypox_model.h5')


In [14]:
mobilenetv2_model.save('mobilenetv2_monkeypox_model.h5')


## Model Evaluation

Evaluating model performance on validation and test datasets.

In [16]:
# Evaluate each model on the test data
resnet_test_loss, resnet_test_acc = resnet_model.evaluate(test_generator)
vgg16_test_loss, vgg16_test_acc = vgg16_model.evaluate(test_generator)
mobilenetv2_test_loss, mobilenetv2_test_acc = mobilenetv2_model.evaluate(test_generator)

print(f'ResNet50 Test accuracy: {resnet_test_acc:.2f}')
print(f'VGG16 Test accuracy: {vgg16_test_acc:.2f}')
print(f'MobileNetV2 Test accuracy: {mobilenetv2_test_acc:.2f}')


22/22 [==============================] - 8s 350ms/step - loss: 0.2487 - accuracy: 0.9137
ResNet50 Test accuracy: 0.68
VGG16 Test accuracy: 0.71
MobileNetV2 Test accuracy: 0.91
